# GPU

查看显卡信息

In [29]:
!nvidia-smi

Thu Jul 20 14:20:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 30%   26C    P8    21W / 350W |    366MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:B1:00.0 Off |                  N/A |
| 30%   

计算设备

In [30]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

查询可用gpu的数量

In [31]:
torch.cuda.device_count()

2

这两个函数允许我们在不存在所需所有GPU的情况下运行代码

In [40]:
def try_gpu(i=0):  
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0), device(type='cuda', index=1)])

查询张量所在的设备

In [33]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

存储在GPU上

In [34]:
# X = torch.ones(2, 3, device=try_gpu())
X = torch.ones(2, 3, device=torch.device('cuda:1'))
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')

第二个GPU上创建一个随机张量

In [35]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.8728, 0.9307, 0.3887],
        [0.4252, 0.3907, 0.1962]], device='cuda:0')

要计算`X + Y`，我们需要决定在哪里执行这个操作

In [36]:
Z = X.cuda(0)
print(X)
print(Y)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')
tensor([[0.8728, 0.9307, 0.3887],
        [0.4252, 0.3907, 0.1962]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


现在数据在同一个GPU上（`Z`和`Y`都在），我们可以将它们相加

In [37]:
Y + Z

tensor([[1.8728, 1.9307, 1.3887],
        [1.4252, 1.3907, 1.1962]], device='cuda:0')

In [38]:
Z.cuda(0) is Z

True

神经网络与GPU

In [44]:
net = nn.Sequential(nn.Linear(3, 1))
# net = net.to(device=try_gpu(1))
net=net.to('cuda:1')

net(X)

tensor([[-0.1001],
        [-0.1001]], device='cuda:1', grad_fn=<AddmmBackward0>)

确认模型参数存储在同一个GPU上

In [46]:
net[0].weight.data.device

device(type='cuda', index=1)